In [5]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

def NaN_rep(data):
    nan=data.isnull().sum()
    print('Number of missing value from each variable:')
    print(nan)
    imputer=KNNImputer(n_neighbors=3)
    nn=imputer.fit_transform(data)
    impdata=pd.DataFrame(nn, columns=data.columns)

    return impdata

In [6]:
def RT_invert(data):
    
    RT_mask = (data.columns.str.contains('RT') | data.columns.str.contains('VSPLOT_OFF'))
    data.loc[:, RT_mask] *=-1
    print(data.loc[:, RT_mask].columns)
    
    return data

In [11]:
def split(data):
# Replace age ranges with numerical values
    age_map = {'22-25': 23, '26-30': 29, '31-35': 33, '36+': 36}
    data['Age_cat'] = data['Age'].map(age_map)

    train_df = pd.concat([group.iloc[:len(group)//2] for name, group in data.groupby(['Gender', 'Age_cat'])])
    test_df = data.drop(train_df.index)
    #drop the extra column
    train_df.drop('Age_cat',axis=1,inplace=True)
    test_df.drop('Age_cat',axis=1,inplace=True)
    return train_df,test_df

In [16]:
def Z_Score(data,plot):
    
    mean = data.mean().values
    sd = data.std().values
    data_norm = ((data-mean)/sd)
    #plot
    if plot:
        fig = plt.figure()
        data_norm.boxplot(rot=90)
        plt.xticks(rotation=90)
        plt.show()
    return data_norm

In [3]:
def outliers_filter(data, plot, threshold):
    
    #mean = data.mean().values
    #sd = data.std().values
    #data_norm = ((data-mean)/sd)
    outlier_mask = (np.abs(data) > threshold).any(axis=1)
    df_clean = data[~outlier_mask]
    if plot:
        df_clean.boxplot(rot=90)
        plt.xticks(rotation=90)
        plt.show()
    return df_clean

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = dataframe.columns
    vif_data["VIF"] = [variance_inflation_factor(dataframe.values, i) for i in range(len(dataframe.columns))]
    return vif_data


In [4]:
#%% regressing out age and gender
from sklearn.linear_model import LinearRegression
def regress_demo(x, data):
    
    #mean = data.mean().values
    #sd = data.std().values
    #y = ((data-mean)/sd)
    # fit regression model
    y=data
    reg = LinearRegression().fit(x,y)
    print(reg.score(x, y))
    # predict values based on model
    pred = reg.predict(x)
    data_res = y-pred
    return data_res